# Create new geotiff by rastering a geojson file

This notebook outlines the steps for rasterising a geojson file, or any other vectors that can be loaded into QGIS.

Command-line steps for this do exist but it's longer and looks more complicated than just using QGIS.

## Setup

In [1]:
# For importing geojson:
import json
# For converting the tif to cog:
import leafmap
# For importing the cog raster image:
import rasterio
# For list searches:
import numpy as np

## 1: Open the geojson file in QGIS

These screenshots show the file `LSOA_(Dec_2011)_Boundaries_Super_Generalised_Clipped_(BSC)_EW_V3.geojson`.

![](images/QGIS_LSOAs.png)

## 2. Open the raster converter

In the top bar, select `Raster > Conversion > Rasterize (Vector to Raster)`.

![](images/QGIS_raster_defaults.png)

## 3. Set rasterization parameters

1. `Field to use for a burn-in value [optional]` 
    1. This should be set if you want an output tiff with more data than just valid/invalid, e.g. every pixel containing the sea is set to 0 and every pixel containing an LSOA is set to 1.
    1. Setting it to `OBJECTID` makes the output file have a different integer for each LSOA. Pixels outside the LSOAs are set to 0.
    1. The integers are given in the same order as the features are contained in the .geojson.
2. `A fixed value to burn [optional]`
    1. When the previous field is set, this option seems to be irnoged.
3. `Output raster size units`
    1. For me, this only works when set to `Pixels` and not when set to `Georeferenced units`.
4. `Width/Horizontal resolution`
    1. The width in pixels of the output image.
5. `Height/Vertical resolution`
    1. The height in pixels of the output image.
6. `Output extent`
    1. The bounds of the map in coordinates. 
    1. This is best set using the options under the `...` drop-down on the right. If it's not there, try making the window wider.
    1. Under the `...` option, `Use map canvas extent` automatically fills in the coordinates and the name of the coordinate system used.
    1. If QGIS is zoomed in on the full map, one of these options will make the output image cover only the zoomed-in-on region.
7. `Assign a specified nodata value to output bands [optional]`
    1. It says optional, but if you don't change it from the default 0, it will use the default 0.
8. `Advanced parameters`
    1. ???
9. `Rasterized`
    1. Choose between saving the output as a temporary file that will be loaded into QGIS immediately and deleted when QGIS is closed, or saving the output as a proper file.
10. `GDAL/OGR console call`
    1. Automatically fills in as you complete the above sections.
    
![](images/QGIS_raster_settings.png)

## 4. Run the rasterisation script

When everything is set, press `Run` and the window will switch to the `Log` tab. If it runs without errors (red text), all is well. Otherwise, mess with the settings until it's happy.

![](images/QGIS_raster_log.png)

## 5. Check the output

The output TIF file should have automatically been loaded into QGIS. In the following screenshot, the integers given to each LSOA are mapped to a greyscale colourmap where 0 is invalid, lower numbers are darker and higher numbers are lighter. 

It looks like whole counties are the same colour because the LSOAs are ordered by LSOA 2011 code in the input geojson file and for the most part neighbouring LSOAs have sequential codes, so the difference in colour is tiny.

![](images/QGIS_raster_output.png)

# Conversion to cloud-optimised geotiff (cog)

With the settings used above (output image 3600 pixels wide by 7200 pixels tall), the output .TIF is a whopping 100MB. That will cause a noticeable lag when loaded into a folium map on Streamlit, for example.

To improve the map load times, the file can be converted to a cloud-optimised geotiff. This is easily done in only a few lines of python by following the instructions [here](https://leafmap.org/notebooks/42_create_cog/).

In [2]:
# Tiff input file name:
in_tiff = './LSOA_raster2.tif'
# Cog output file name:
out_cog = 'LSOA_cog2.tif'

leafmap.image_to_cog(in_tiff, out_cog)

Reading input: /home/anna/samuel_book/skeleton-pathway-model/region_matching/LSOA_raster2.tif

Adding overviews...
Updating dataset tags...
Writing output to: /home/anna/samuel_book/skeleton-pathway-model/region_matching/LSOA_cog2.tif


If the following output is `(True, [], [])` then the conversion was successful.

In [3]:
leafmap.cog_validate(out_cog)

(True, [], [])

The size of the resulting cog file is considerably smaller than the input tiff at only 2.1MB.

# Import the cog file into python

Just a quick check that the tiff was saved at high enough resolution to get all of the information we need from the original geojson vectors. In particular,  I would like each LSOA to contribute at least one pixel to the output image. This condition would not be met by the small London at a low output image resolution.

Import the .geojson and see how many LSOAs it contains:

In [4]:
geojson_file = './LSOA_(Dec_2011)_Boundaries_Super_Generalised_Clipped_(BSC)_EW_V3.geojson'

with open(geojson_file) as f:
    geojson_ew = json.load(f)

The property saved into the geotiff is `OBJECTID` so make a list of this and some other useful bits.

In [5]:
areas = []
LSOA_names = []
LSOA_codes = []
object_IDs = []
for i, feature in enumerate(geojson_ew['features']):
    areas.append(feature['properties']['Shape__Area'])
    LSOA_names.append(feature['properties']['LSOA11NMW'])
    LSOA_codes.append(feature['properties']['LSOA11CD'])
    object_IDs.append(feature['properties']['OBJECTID'])

areas = np.array(areas)
LSOA_names = np.array(LSOA_names)
LSOA_codes = np.array(LSOA_codes)
object_IDs = np.array(object_IDs)

Import the cog file and pull out its unique values.

In [6]:
with rasterio.open(out_cog, 'r') as f:
    tif_array = f.read()

tif_vals = sorted(list( set( tif_array.ravel().tolist() )))

See which values are in the tiff but not in the geojson. This should return just the value 0, which is given to invalid data in the tiff.

In [7]:
set( tif_array.ravel().tolist() ) - set(object_IDs)

{0.0}

See which values are in the geojson but not in the tiff. These are the object IDs of LSOAs that are present in the geojson but not represented in the tiff.

In [8]:
missing_LSOAs = list(set(object_IDs) - set( tif_array.ravel().tolist() ))

missing_LSOAs

[32580, 32581, 4550, 32237, 31950, 32181, 18684, 8286]

Find the names of these missing LSOAs:

In [9]:
for LSOA in missing_LSOAs:
    print(LSOA, LSOA_names[np.where(object_IDs == LSOA)])

32580 ['Islington 011E']
32581 ['Islington 011F']
4550 ['Westminster 024A']
32237 ['Haringey 020F']
31950 ['Tower Hamlets 031F']
32181 ['Hounslow 025G']
18684 ['Carlisle 001B']
8286 ['North Tyneside 012C']
